In [1]:
import requests
from dotenv import load_dotenv
import os
import pandas as pd
import torch
load_dotenv()
from sentence_transformers.util import semantic_search

model_id = os.getenv("model_id")
hf_token = os.getenv("hf_token")
api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
headers = {"Authorization": f"Bearer {hf_token}"}


def getEmbeddings(texts):
    response = requests.post(api_url, headers=headers, json={"inputs": texts, "options":{"wait_for_model":True}})
    embeddedData = response.json()
    return torch.FloatTensor(embeddedData)



c:\Users\user\Desktop\CTV\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
texts = ["How do I get a replacement Medicare card?",
        "What is the monthly premium for Medicare Part B?",
        "How do I terminate my Medicare Part B (medical insurance)?",
        "How do I sign up for Medicare?",
        "Can I sign up for Medicare Part B if I am working and have health insurance through an employer?",
        "How do I sign up for Medicare Part B if I already have Part A?",
        "What are Medicare late enrollment penalties?",
        "What is Medicare and who can get it?",
        "How can I get help with my Medicare Part A and Part B premiums?",
        "What are the different parts of Medicare?",
        "Will my Medicare premiums be higher because of my higher income?",
        "What is TRICARE ?",
        "Should I sign up for Medicare Part B if I have Veterans' Benefits?"]
# dataset_embeddings = getEmbeddings(texts)


questions = ["How can Medicare help me?", 
            "What are the benefits of Insurance", 
            "What is Medicare and who can get it?"]
# query_embeddings = getEmbeddings(question)


In [8]:
def mis_matches(dataset, queryset, threshold=0.9):
    dataset_embeddings = getEmbeddings(dataset)
    query_embeddings = getEmbeddings(queryset)
    ans = []
    for index, embedding in enumerate(query_embeddings):
        hits = semantic_search(embedding, dataset_embeddings, top_k=1)
        print(index, hits)
        # if hits[0][0]['score']>threshold:
        #     ans.append([index, hits[0][0]['corpus_id'], hits[0][0]['score']])
        #     print([dataset[hits[i]['corpus_id']] for i in range(len(hits))])
    # print("Ans:", ans)

mis_matches(texts, questions, 0.5)

0 [[{'corpus_id': 8, 'score': 0.7565304636955261}]]
1 [[{'corpus_id': 9, 'score': 0.3491109013557434}]]
2 [[{'corpus_id': 7, 'score': 1.0000001192092896}]]


In [11]:
def mis_matches(dataset, queryset, threshold=0.9):
    dataset_embeddings = getEmbeddings(dataset)
    query_embeddings = getEmbeddings(queryset)
    hits = semantic_search(query_embeddings, dataset_embeddings, top_k=1)
    ans = [
        [index, hit[0]['corpus_id'], hit[0]['score']] 
        for index, hit in enumerate(hits)
        if hit[0]['score']>threshold
    ]
    # print(ans)
    return ans

mis_matches(texts, questions, 0.8)

[[2, 7, 1.0000001192092896]]
